In [1]:
import typing as T  # pylint: disable=unused-import

import numpy as np
import numpy.typing as npt

from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module, unused-import
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
    MosekSolver,
    MosekSolverDetails,
    SolverOptions,
    CommonSolverOption,
)
from pydrake.geometry.optimization import (  # pylint: disable=import-error, no-name-in-module
    HPolyhedron,
    Point,
    ConvexSet,
)
from pydrake.symbolic import ( # pylint: disable=import-error, no-name-in-module, unused-import
    Polynomial,
    Variable,
    Variables,
    Expression,
)  
from program_options import ProgramOptions, FREE_POLY, PSD_POLY, NOISY_DEG_2

from util import (
    timeit,
    diditwork,
    INFO,
    YAY,
    WARN,
    ERROR,
)  # pylint: disable=import-error, no-name-in-module, unused-import

In [5]:
potential = Polynomial(0)
prog = MathematicalProgram()
x = prog.NewContinuousVariables(3)
y = np.zeros(3)
potential.EvaluatePartial(
            {x[i]: y[i] for i in range(3)}
        )

<Polynomial "0">

In [4]:
triu_ind = np.triu_indices(3, k=1)
A = np.array([[1,2,3],[4,5,6],[7,8,9]])
list(A[triu_ind[0], triu_ind[1]])

[2, 3, 6]

In [5]:
x = np.ones(3)
Q = np.eye(3)
x.dot(Q)

array([1., 1., 1.])

In [6]:
x

array([1., 1., 1.])

In [1]:
import typing as T
import numpy.typing as npt

import numpy as np

from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
) 
from plotly.express.colors import sample_colorscale
import plotly.graph_objects as go 

import logging

from pydrake.math import le, eq # pylint: disable=import-error, no-name-in-module
from pydrake.symbolic import Polynomial, Variable, Variables, Expression, EvenDegreeMonomialBasis # pylint: disable=import-error, no-name-in-module
from pydrake.math import eq, le, ge # pylint: disable=import-error, no-name-in-module
from pydrake.geometry.optimization import ConvexSet, VPolytope, HPolyhedron, GraphOfConvexSets, GraphOfConvexSetsOptions, Spectrahedron, Point # pylint: disable=import-error, no-name-in-module
from pydrake.solvers import MakeSemidefiniteRelaxation # pylint: disable=import-error, no-name-in-module

from IPython.display import Markdown, display 

from util import timeit, diditwork, ERROR, INFO, WARN, YAY, have_full_dimensional_intersection, latex
from gcs_util import get_solution_values

In [2]:
def get_polyhedral_bezier_set(hpoly:HPolyhedron, num_control_points:int):
    def make_diag_from_mat(mat1, mat2):
        # constructs the following matrix:
        # | mat1  0   |
        # | 0    mat2 |
        row1 = np.hstack( (mat1, np.zeros((mat1.shape[0], mat2.shape[1])) ) )
        row2 = np.hstack( (np.zeros((mat2.shape[0], mat1.shape[1])), mat2 ) )
        return np.vstack((row1, row2))

    A,b = hpoly.A(), hpoly.b()
    # construct set matrix for bezier curve -- k points, for each Ax <= b
    bezier_A, bezier_b = A,b
    for i in range(num_control_points-1):
        bezier_A = make_diag_from_mat(bezier_A, A)
        bezier_b = np.hstack((bezier_b, b))
    return HPolyhedron(bezier_A, bezier_b)

In [11]:
def get_spectrahedral_bezier_set(hpoly:HPolyhedron, num_control_points:int):
    state_dim = hpoly.A().shape[1]
    # convert vpoly to hpoly
    bezier_hpoly = get_polyhedral_bezier_set(hpoly, num_control_points)
    A, b = bezier_hpoly.A(), bezier_hpoly.b()
    # define the program
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(num_control_points*state_dim, "x")
    for i in range(len(b)):
        prog.AddLinearConstraint( A[i].dot(x) <= b[i] )
    # construct SOS relaxation
    sdp_prog = MakeSemidefiniteRelaxation(prog)
    # extract spectrahedral set from it
    return Spectrahedron(sdp_prog)
    

In [5]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(3, "x")
sdp_prog = MakeSemidefiniteRelaxation(prog)
latex(sdp_prog)
sdp_prog.decision_variables()

\begin{align*}
\text{find}_{x_{0}, x_{1}, x_{2}, Y_{0,0}, Y_{1,0}, Y_{2,0}, Y_{1,1}, Y_{2,1}, Y_{2,2}, one} \quad & \\
 \text{subject to}\quad & one = 1,\\
 & \begin{bmatrix} Y_{0,0} & Y_{1,0} & Y_{2,0} & x_{0} \\ Y_{1,0} & Y_{1,1} & Y_{2,1} & x_{1} \\ Y_{2,0} & Y_{2,1} & Y_{2,2} & x_{2} \\ x_{0} & x_{1} & x_{2} & one \end{bmatrix} \succeq 0.\\
\end{align*}


array([Variable('x(0)', Continuous), Variable('x(1)', Continuous),
       Variable('x(2)', Continuous), Variable('Y(0,0)', Continuous),
       Variable('Y(1,0)', Continuous), Variable('Y(2,0)', Continuous),
       Variable('Y(1,1)', Continuous), Variable('Y(2,1)', Continuous),
       Variable('Y(2,2)', Continuous), Variable('one', Continuous)],
      dtype=object)

In [13]:
def get_noisy_spectrahedral_bezier_set(hpoly:HPolyhedron, num_control_points:int, noise_variance: npt.NDArray):
    state_dim = hpoly.A().shape[1]
    assert noise_variance.shape == (state_dim, state_dim)
    def make_diag_from_mat(mat1, mat2):
        # constructs the following matrix:
        # | mat1  0   |
        # | 0    mat2 |
        row1 = np.hstack( (mat1, np.zeros((mat1.shape[0], mat2.shape[1])) ) )
        row2 = np.hstack( (np.zeros((mat2.shape[0], mat1.shape[1])), mat2 ) )
        return np.vstack((row1, row2))
    bezier_noise_variance = make_diag_from_mat(noise_variance, np.zeros((1,1)) )
    for i in range(num_control_points-1):
        bezier_noise_variance = make_diag_from_mat(noise_variance, bezier_noise_variance)
    
    # get big constraint matrix
    bezier_hpoly = get_polyhedral_bezier_set(hpoly, num_control_points)
    A, b = bezier_hpoly.A(), bezier_hpoly.b()
    B = np.hstack( (b.reshape((len(b),1)), -A) )

    # define the program
    prog = MathematicalProgram()
    bezier_state_dim = num_control_points*state_dim
    x = prog.NewContinuousVariables(bezier_state_dim, "x")
    X = prog.NewSymmetricContinuousVariables(bezier_state_dim, "Y")
    one = prog.NewContinuousVariables(1, "one")[0]
    prog.AddLinearConstraint(one == 1)

    big_mat = np.vstack((np.hstack((X, x.reshape((bezier_state_dim,1)))), np.hstack((x, [one])) ))

    # moment 1 constraints
    for i in range(len(b)):
        prog.AddLinearConstraint( A[i].dot(x) <= b[i]*one )

    # moment 2 constraints
    constraints = ge(B.dot(big_mat).dot(B.T), 0)
    triu_ind = np.triu_indices(constraints.shape[0], k=1)
    unique_constraints = constraints[triu_ind[0], triu_ind[1]]
    for con in unique_constraints:
        prog.AddLinearConstraint(con)
    prog.AddPositiveSemidefiniteConstraint(big_mat - bezier_noise_variance)
    INFO(big_mat - bezier_noise_variance)
    latex(prog)
    INFO(prog.decision_variables())
    # extract spectrahedral set from it
    return Spectrahedron(sdp_prog)

get_noisy_spectrahedral_bezier_set(HPolyhedron.MakeBox([0],[1]), 2, np.eye(1)*0.1)

[[<Expression "(-0.10000000000000001 + Y(0,0))"> <Expression "Y(1,0)">
  <Expression "x(0)">]
 [<Expression "Y(1,0)"> <Expression "(-0.10000000000000001 + Y(1,1))">
  <Expression "x(1)">]
 [<Expression "x(0)"> <Expression "x(1)"> <Expression "one(0)">]]


\begin{align*}
\text{find}_{x_{0}, x_{1}, Y_{0,0}, Y_{1,0}, Y_{1,1}, one_{0}, Symmetric_{0,0}, Symmetric_{1,0}, Symmetric_{2,0}, Symmetric_{1,1}, Symmetric_{2,1}, Symmetric_{2,2}} \quad & \\
 \text{subject to}\quad & (x_{0} - one_{0}) \le 0,\\
 & (x_{1} - one_{0}) \le 0,\\
 & ( - Y_{1,0} + Y_{1,1}) \le 0,\\
 & (x_{0} - x_{1} - Y_{0,0} + Y_{1,0}) \le 0,\\
 & ( - x_{0} + x_{1}) \le 0,\\
 & (x_{1} - Y_{1,0}) \le 0,\\
 & ( - x_{0} + one_{0}) \le 0,\\
 & one_{0} = 1,\\
 & \begin{bmatrix} 1 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 1 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 1 & -1 & 0 & 0 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 1 & -1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & -1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & -1 \end{bmatrix} \begin{bmatrix} Y_{0,0} \\ Symmetric_{0,0} \\ Y_{1,0} \\ Symmetric_{1,0} \\ x_{0} \\ Symmetric_{2,0} \\ Y_{1,1} \\ Symmetric_{1,1} \\ x_{1} \\ Symmetric_{2,1} \\ one_{0} \\ Symmetric_{2,2} \end{bmatrix} = \begin{bmatrix} 0.100 \\ 0 \\ 0 \\ 0.100 \\ 0 \\ 0 \end{bmatrix},\\
 & 0 \le x_{0},\\
 & 0 \le x_{1},\\
 & -0 \le x_{1},\\
 & \begin{bmatrix} Symmetric_{0,0} & Symmetric_{1,0} & Symmetric_{2,0} \\ Symmetric_{1,0} & Symmetric_{1,1} & Symmetric_{2,1} \\ Symmetric_{2,0} & Symmetric_{2,1} & Symmetric_{2,2} \end{bmatrix} \succeq 0.\\
\end{align*}


[Variable('x(0)', Continuous) Variable('x(1)', Continuous)
 Variable('Y(0,0)', Continuous) Variable('Y(1,0)', Continuous)
 Variable('Y(1,1)', Continuous) Variable('one(0)', Continuous)
 Variable('Symmetric(0,0)', Continuous)
 Variable('Symmetric(1,0)', Continuous)
 Variable('Symmetric(2,0)', Continuous)
 Variable('Symmetric(1,1)', Continuous)
 Variable('Symmetric(2,1)', Continuous)
 Variable('Symmetric(2,2)', Continuous)]


In [2]:
dim = 3
int(dim*(dim+1)/2)

6